In [ ]:
pip install pandas scikit-learn spacy pyttsx3 SpeechRecognition sounddevice
python3 -m spacy download en_core_web_sm


Defaulting to user installation because normal site-packages is not writeable
/opt/homebrew/opt/python@3.13/bin/python3.13: No module named spacy


In [ ]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pyttsx3
import speech_recognition as sr
import sounddevice as sd
import wavio
import re
import numpy as np


/Users/anchalbhonekar/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
df = pd.read_csv("/Users/anchalbhonekar/Desktop/data.csv")

In [ ]:
stop_words = set(['a','an','the','and','or','is','are','of','to','in','for','on','with','you','we','our','your'])

def preprocess(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = [t.lower() for t in text.split() if t.lower() not in stop_words]
    return " ".join(tokens)

df['processed'] = df['Question'].apply(preprocess)


In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['processed'])

def get_response(user_input):
    user_input_processed = preprocess(user_input)
    user_vec = vectorizer.transform([user_input_processed])
    similarity = cosine_similarity(user_vec, X)
    return df['Answer'].iloc[similarity.argmax()]

In [ ]:
nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents]

In [ ]:
engine = pyttsx3.init()

In [ ]:
recognizer = sr.Recognizer()
fs = 16000  
duration = 5  

In [ ]:
print("Voice Chatbot ready! Say 'quit' to exit.")

while True:
    print("Listening...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()
    wavio.write("temp.wav", recording, fs, sampwidth=2)

    with sr.AudioFile("temp.wav") as source:
        audio = recognizer.record(source)
    
    try:
        user_input = recognizer.recognize_google(audio)
        print("You:", user_input)
        
        if user_input.lower() == "quit":
            print("Exiting chatbot...")
            break
        
        entities = extract_entities(user_input)
        if entities:
            print("Entities detected:", entities)
        
        response = get_response(user_input)
        print("Bot:", response)
        
        engine.say(response)
        engine.runAndWait()

    except sr.UnknownValueError:
        print("Sorry, I didn't catch that.")
    except sr.RequestError as e:
        print(f"Could not request results; {e}")

Voice Chatbot ready! Say 'quit' to exit.
Listening...
Sorry, I didn't catch that.
Listening...
You: what are your
Bot: You can return products within 30 days of purchase with a receipt, as long as the items are unused and in original packaging.
Listening...
You: where is your store located
Bot: Our main store is located at [Your Address]. You can also check our website for other locations.
Listening...
You: quit
Exiting chatbot...
